In [1]:
import os
os.chdir('/Users/francopiccolo/GitHub/learn-to-rank')

In [2]:
import pandas as pd
X_test = pd.read_pickle('./data/X_test.pkl')
X_val = pd.read_pickle('./data/X_val.pkl')
y_test = pd.read_pickle('./data/y_test.pkl')
y_val = pd.read_pickle('./data/y_val.pkl')
test_query_sizes = pd.read_pickle('./data/test_query_sizes.pkl')
val_query_sizes = pd.read_pickle('./data/val_query_sizes.pkl')

In [3]:
X_test.columns

Index(['device_type', 'language', 'nr_guests', 'property_type',
       'distance_to_center', 'dorm_price', 'private_price', 'security_score',
       'location_score', 'staff_score', 'fun_score', 'cleanliness_score',
       'value_for_money_score', 'facilities_score'],
      dtype='object')

In [4]:
import lightgbm as lgb
bst = lgb.Booster(model_file='./data/model.txt')

In [5]:
from sklearn.metrics import ndcg_score

In [6]:
def calculate_ndcg(test_query_sizes, X_test, y_test):
    i = 0
    ndcg_cum = 0
    queries = 0
    for query_size in test_query_sizes:
        y_true = y_test[i:i+query_size].values.reshape(1, -1)
        if query_size == 1 or y_true.sum() == 0:
            ndcg_cum += 1
        else:
            y_pred = bst.predict(X_test[i:i+query_size]).reshape(1, -1)
            ndcg = ndcg_score(y_true, y_pred, k=5)
            ndcg_cum += ndcg
        queries += 1
        i += query_size            
    return ndcg_cum, queries

In [ ]:
ndcg_cum_val, queries_val = calculate_ndcg(val_query_sizes, X_val, y_val)
ndcg_cum_test, queries_test = calculate_ndcg(test_query_sizes, X_test, y_test)

In [ ]:
ndcg_val = ndcg_cum_val/queries_val
ndcg_test = ndcg_cum_test/queries_test

In [ ]:
print(ndcg_val, ndcg_test)

In [ ]:
bst.feature_name()